## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-24-14-39-41 +0000,nypost,Trump announces he’ll attend Ryder Cup in New ...,https://nypost.com/2025/08/24/us-news/trump-an...
1,2025-08-24-14-38-15 +0000,bbc,"More than 30 jihadists killed in air strikes, ...",https://www.bbc.com/news/articles/cq58123z814o...
2,2025-08-24-14-34-38 +0000,bbc,"More than 500,000 ordered to evacuate as typho...",https://www.bbc.com/news/articles/cx2pw5ypwqeo...
3,2025-08-24-14-33-03 +0000,nyt,China’s Tropical Resort Island Orders Evacuati...,https://www.nytimes.com/2025/08/24/world/asia/...
4,2025-08-24-14-26-21 +0000,bbc,Three sisters drown in migrant boat in Mediter...,https://www.bbc.com/news/articles/cp89rqgjq1no...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2322/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
0,trump,22
7,new,11
43,ukraine,9
81,russia,7
51,all,6


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
0,2025-08-24-14-39-41 +0000,nypost,Trump announces he’ll attend Ryder Cup in New ...,https://nypost.com/2025/08/24/us-news/trump-an...,52
133,2025-08-23-17-43-00 +0000,wsj,Chicago will likely be the next target in what...,https://www.wsj.com/politics/policy/trump-look...,48
20,2025-08-24-13-00-00 +0000,wsj,The New Acronym Driving South Korea’s Summit W...,https://www.wsj.com/world/asia/the-new-acronym...,47
44,2025-08-24-10-00-00 +0000,latimes,California's lightning-fast push for partisan ...,https://www.latimes.com/politics/story/2025-08...,45
71,2025-08-24-04-01-16 +0000,nyt,China Looms Large as South Korea’s President M...,https://www.nytimes.com/2025/08/24/world/asia/...,44


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
0,52,2025-08-24-14-39-41 +0000,nypost,Trump announces he’ll attend Ryder Cup in New ...,https://nypost.com/2025/08/24/us-news/trump-an...
39,42,2025-08-24-10-24-22 +0000,nypost,Russia accuses Kyiv of striking nuclear power ...,https://nypost.com/2025/08/24/world-news/russi...
80,32,2025-08-24-02-00-00 +0000,wsj,Bashing the president’s tariff policies could ...,https://www.wsj.com/politics/policy/gop-tariff...
83,31,2025-08-24-01-14-02 +0000,nypost,"Rikers Island inmate found dead in cell, marki...",https://nypost.com/2025/08/23/us-news/rikers-i...
103,29,2025-08-23-21-52-29 +0000,nypost,Israel says it will begin Gaza City military t...,https://nypost.com/2025/08/23/world-news/israe...
118,27,2025-08-23-20-15-36 +0000,nypost,"Putin war hero, the ‘Executioner,’ exposed as ...",https://nypost.com/2025/08/23/world-news/putin...
65,23,2025-08-24-05-48-50 +0000,nypost,Rescue effort halted for stuck climber on trea...,https://nypost.com/2025/08/24/world-news/kyrgy...
40,21,2025-08-24-10-00-27 +0000,nypost,Zohran Mamdani wants to end all misdemeanor ch...,https://nypost.com/2025/08/24/us-news/nyc-mayo...
12,21,2025-08-24-14-05-19 +0000,nypost,Russia’s foreign minister throws up a new cond...,https://nypost.com/2025/08/24/us-news/russias-...
114,21,2025-08-23-20-47-34 +0000,nyt,Abrego Garcia’s Lawyers Say U.S. Officials Tri...,https://www.nytimes.com/2025/08/23/us/politics...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
